# Elliptic Curves over the Reals

**Module 06** | 06-elliptic-curves

*Plotting y^2 = x^3 + ax + b*

## Objectives

By the end of this notebook you will be able to:

1. Understand the core ideas behind **elliptic curves over the reals**.
2. Explore these concepts interactively using SageMath.
3. Build intuition through hands-on computation and visualization.

## Prerequisites

- Basic familiarity with Python syntax.
- A working SageMath installation (or access to CoCalc/SageMathCell).

## The Weierstrass Equation

In [ ]:
# An elliptic curve: y^2 = x^3 + ax + b
# The discriminant must be nonzero: 4a^3 + 27b^2 != 0
a, b = -1, 1
print(f'Discriminant: {4*a^3 + 27*b^2}')

## Plotting Curves

In [ ]:
# Plot an elliptic curve over the reals
E = EllipticCurve(RR, [-1, 1])
E.plot()

## Varying Parameters

In [ ]:
# Explore how a and b affect the curve shape
# TODO: plot several curves with different a, b values

## Exercises

Try these on your own before moving on:

1. **Exercise 1:** *(TODO: add exercise)*
2. **Exercise 2:** *(TODO: add exercise)*
3. **Exercise 3:** *(TODO: add exercise)*

## Summary

In this notebook we explored **elliptic curves over the reals**.  Key takeaways:

- *(TODO: summarize key point 1)*
- *(TODO: summarize key point 2)*
- *(TODO: summarize key point 3)*

**Next:** [Point Addition: The Geometry](06b-point-addition-geometry.ipynb)